# 🤖 08_Final_Training — Фінальне донавчання моделі LightGBM
# Project: Прогнозування Відтоку Клієнтів
# Team Lead: Vitalii Subbotin
# Date: 2025-10-16

In [8]:
# 1. Імпорт бібліотек
import pandas as pd
import numpy as np
import joblib
from pathlib import Path
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
# 2. Шляхи
BASE_DIR = Path("..")
DATA_PATH = BASE_DIR / "data" / "df1_mean.csv"
MODELS_DIR = BASE_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print("📂 DATA_PATH ->", DATA_PATH)

📂 DATA_PATH -> ..\data\df1_mean.csv


In [10]:
# 3. Завантаження та базова обробка даних
df = pd.read_csv(DATA_PATH)

if 'id' in df.columns:
    df.drop(columns=['id'], inplace=True)
if 'Churn' in df.columns and 'churn' not in df.columns:
    df.rename(columns={'Churn': 'churn'}, inplace=True)

In [11]:
# 4. Перетворення категорій на 0/1
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].replace({'Так': 1, 'Ні': 0}).astype(int)

X = df.drop(columns=['churn'])
y = df['churn']

In [12]:
# 5. Holdout split
X_trainval, X_holdout, y_trainval, y_holdout = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print(f"✅ Train+Val shape: {X_trainval.shape} | Holdout shape: {X_holdout.shape}")

✅ Train+Val shape: (57514, 10) | Holdout shape: (14379, 10)


In [13]:
# 6. Масштабування всіх числових ознак
scaler = StandardScaler()
scaler.fit(X_trainval)

X_trainval_scaled = pd.DataFrame(scaler.transform(X_trainval), columns=X_trainval.columns)
X_holdout_scaled = pd.DataFrame(scaler.transform(X_holdout), columns=X_holdout.columns)

In [14]:
# 7. Модель LightGBM
best_params = {
    'n_estimators': 300,
    'learning_rate': 0.05,
    'max_depth': -1,
    'num_leaves': 63,
    'subsample': 0.8,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
}

model = LGBMClassifier(**best_params)
model.fit(X_trainval_scaled, y_trainval)

print("✅ Модель LightGBM донавчено на повному train+val наборі.")

✅ Модель LightGBM донавчено на повному train+val наборі.


In [15]:
# 8. Збереження артефактів
joblib.dump(model, MODELS_DIR / "best_model_lgb.pkl")
joblib.dump(scaler, MODELS_DIR / "scaler.pkl")

X_holdout_scaled.to_csv(MODELS_DIR / "X_holdout.csv", index=False)
y_holdout.to_csv(MODELS_DIR / "y_holdout.csv", index=False)

print("💾 Збережено артефакти у папку:", MODELS_DIR)
print("📦 Скалер тепер має:", len(scaler.feature_names_in_), "ознак.")
print("✅ Ознаки:", scaler.feature_names_in_)

💾 Збережено артефакти у папку: ..\models
📦 Скалер тепер має: 10 ознак.
✅ Ознаки: ['is_tv_subscriber' 'is_movie_package_subscriber' 'subscription_age'
 'bill_avg' 'reamining_contract' 'service_failure_count' 'download_avg'
 'upload_avg' 'download_over_limit' 'reamining_contract_missing']
